In [ ]:
import numpy as np
from spectral_dl.srcnn import SrCNN 
from spectral_dl.dataset import ARPESDataset, NPYArpes
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
from torchvision.transforms.transforms import ToPILImage
import torchvision.transforms as transforms


In [ ]:
folder_ = "./FS_nigth2"
path2weights = './spectral_dl/weights.pt'
learning_rate = 5e-4
batch_size = 1
flux = 1e3
alpha = 0.7

In [ ]:
dset = ARPESDataset(folder_, flux)
dl = DataLoader(dset, batch_size, shuffle=True,)

In [ ]:
dset = NPYArpes(['./train_bisn_data.npy', './train_bisn_noise.npy'])
dl = DataLoader(dset, batch_size, shuffle=True,)

In [ ]:
net = SrCNN().cuda()
net.load_state_dict(torch.load(path2weights))
net.eval()

In [ ]:
outputs = []
noisy = []
clean = []
net.eval()
with torch.no_grad():
    for (noise, data) in tqdm(dl):
        noise = noise.cuda(non_blocking=True).to(torch.float32)
        data = data.cuda(non_blocking=True).to(torch.float32)
        out = net(noise.unsqueeze(1))
        outputs.append(out.squeeze(1).detach().cpu().numpy())
        noisy.append(noise.squeeze(1).detach().cpu().numpy())
        clean.append(data.squeeze(1).detach().cpu().numpy())

In [ ]:
outputs = np.array(outputs)

In [ ]:
noisy = np.array(noisy)

In [ ]:
clean = np.array(clean)

In [ ]:
data = data.detach().cpu().numpy()
noise = noise.detach().cpu().numpy()
out = out.detach().cpu().numpy()

In [ ]:
outputs.shape

In [ ]:
outputs = np.squeeze(outputs, axis=1)

In [ ]:
noisy = np.squeeze(noisy, axis=1)

In [ ]:
outputs.shape

In [ ]:
noisy.shape

In [ ]:
np.save('outputs.npy', outputs)

In [ ]:
np.save('noisy.npy', noisy)

In [ ]:
data.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(clean[100,0,10,:])

In [ ]:
plt.plot(noisy[100,10,:])

In [ ]:
plt.plot(outputs[100,10,:])

In [ ]:
import ipyvolume as ipv

In [ ]:
outputs.shape

In [ ]:
plt.imshow(outputs[100,:,:])

In [ ]:
plt.imshow(noisy[100,:,:])

In [ ]:
plt.imshow(clean[100,0,:,:])

In [ ]:
import math
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return 0
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [ ]:
calculate_psnr(clean[140], noisy[140])

In [ ]:
calculate_psnr(clean[140], outputs[140,0])

In [ ]:
psnr_avg = 0
psnr_noise = 0
for i in range(outputs.shape[0]):
    psnr_avg += calculate_psnr(clean[i,0], outputs[i])
    psnr_noise += calculate_psnr(clean[i,0], noisy[i])

In [ ]:
psnr_avg/outputs.shape[0]

In [ ]:
psnr_noise/outputs.shape[0]

In [ ]:
clean.shape

In [ ]:
data.shape

In [ ]:
out.shape

In [ ]:
noise.shape

In [ ]:
from skimage.metrics import structural_similarity as ssim

In [ ]:
ssim(data[0], out[0,0])

In [ ]:
ssim(data[0], noise[0])

In [ ]:
ssim_avg = 0
ssim_noise = 0
for i in range(outputs.shape[0]):
    if ssim(clean[i,0], noisy[i]) < 1.0:
        ssim_avg += ssim(clean[i,0], outputs[i])
        ssim_noise += ssim(clean[i,0], noisy[i])

In [ ]:
ssim_avg/outputs.shape[0]

In [ ]:
ssim_noise/outputs.shape[0]

In [ ]:
from scipy.ndimage import gaussian_filter, laplace

In [ ]:
plt.imshow(laplace(noisy[100,:,:]))

In [ ]:
plt.imshow(laplace(outputs[100,:,:]))

In [ ]:
plt.imshow(laplace(clean[100,0,:,:]))